# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port mathurin,-19.6833,63.4167,21.96,79,30,6.45,MU,1721776965
1,1,chonchi,-42.6128,-73.8092,8.40,87,96,1.99,CL,1721776966
2,2,waitangi,-43.9535,-176.5597,9.34,99,100,4.92,NZ,1721776967
3,3,ribeira grande,38.5167,-28.7000,23.21,83,4,3.13,PT,1721776967
4,4,adamstown,-25.0660,-130.1015,18.77,65,90,10.59,PN,1721776968


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 0.5,
    color = "City",
    frame_width=600,
    frame_height=400
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) &
                                (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,33,p'ot'i,42.1462,41.6720,22.81,88,0,1.29,GE,1721776985
79,79,isla cristina,37.2000,-7.3167,26.32,48,0,0.69,ES,1721777012
221,221,pacific grove,36.6177,-121.9166,26.80,67,0,3.60,US,1721777096
225,225,buka,40.8108,69.1986,25.62,25,0,1.45,UZ,1721777098
370,370,lata,40.1629,-8.3327,24.95,54,0,0.63,PT,1721777182
379,379,crotone,39.0851,17.1178,26.77,55,0,0.86,IT,1721777187
444,444,hammerfest,70.6634,23.6821,23.25,56,0,2.06,NO,1721777223
488,488,vilhena,-12.7406,-60.1458,22.12,46,0,2.06,BR,1721777249
492,492,puerto iguazu,-25.5991,-54.5736,21.49,81,0,1.54,AR,1721777061
514,514,qubadli,39.3440,46.5820,25.20,51,0,2.57,AZ,1721777264


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
33,p'ot'i,GE,42.1462,41.6720,88,
79,isla cristina,ES,37.2000,-7.3167,48,
221,pacific grove,US,36.6177,-121.9166,67,
225,buka,UZ,40.8108,69.1986,25,
370,lata,PT,40.1629,-8.3327,54,
379,crotone,IT,39.0851,17.1178,55,
444,hammerfest,NO,70.6634,23.6821,56,
488,vilhena,BR,-12.7406,-60.1458,46,
492,puerto iguazu,AR,-25.5991,-54.5736,81,
514,qubadli,AZ,39.3440,46.5820,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    'apiKey': geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, 'Lat']
    lng = hotel_df.loc[index, 'Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
p'ot'i - nearest hotel: No hotel found
isla cristina - nearest hotel: Hotel Isla Cristina Palace
pacific grove - nearest hotel: Pacific Grove Inn
buka - nearest hotel: No hotel found
lata - nearest hotel: Residencial Botânico
crotone - nearest hotel: Palazzo Foti
hammerfest - nearest hotel: Thon Hotel Hammerfest
vilhena - nearest hotel: No hotel found
puerto iguazu - nearest hotel: El Libertador
qubadli - nearest hotel: No hotel found
belmonte - nearest hotel: No hotel found
birjand - nearest hotel: مهمانسرای جهانگردی بیرجند


,City,Country,Lat,Lng,Humidity,Hotel Name
33,p'ot'i,GE,42.1462,41.6720,88,No hotel found
79,isla cristina,ES,37.2000,-7.3167,48,Hotel Isla Cristina Palace
221,pacific grove,US,36.6177,-121.9166,67,Pacific Grove Inn
225,buka,UZ,40.8108,69.1986,25,No hotel found
370,lata,PT,40.1629,-8.3327,54,Residencial Botânico
379,crotone,IT,39.0851,17.1178,55,Palazzo Foti
444,hammerfest,NO,70.6634,23.6821,56,Thon Hotel Hammerfest
488,vilhena,BR,-12.7406,-60.1458,46,No hotel found
492,puerto iguazu,AR,-25.5991,-54.5736,81,El Libertador
514,qubadli,AZ,39.3440,46.5820,51,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    scale = 0.5,
    color = "City",
    frame_width=600,
    frame_height=350,
    hover_cols=["City", "Humidity", "Hotel Name", "Country", "Lng", "Lat"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)